<a href="https://colab.research.google.com/github/stefanbringuier/randomonium/blob/main/notebooks/shape_e_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook to play around with **shape-e**

https://github.com/openai/shap-e

In [ ]:
! git clone https://github.com/openai/shap-e.git
! cd shap-e; pip install -e .

**NOTE: Restart runtime after installing**

In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:05<00:00, 156MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [4]:
batch_size = 4
guidance_scale = 15.0
prompt = "A bionic eye"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [5]:
render_mode = 'nerf' # you can change this to 'stf'
size = 256 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzM2ZmM2YzZjNmZmZmZm…

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYUAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzMwBmMzNmM2ZmMzMzZm…

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzMwBmMzNmM2ZmMwAzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYUAAAAAADMAAAAzADMzAGYzAAAAMzMAMwAzMzMzM2YzM5kzM8wzMz…

In [6]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'bionic_eye_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'bionic_eye_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(
